In [1]:
from decimal import Decimal, getcontext

def Sn_ascending(n):
    S = 0
    for k in range(2, n + 1):
        S += 10**7 / (k**2 - 1)
    return S

S_ascending = Sn_ascending(10**6)
print("S_10^6 (从小到大):", S_ascending)

def Sn_descending(n):
    S = 0
    for k in range(n, 1, -1):
        S += 10**7 / (k**2 - 1)
    return S

S_descending = Sn_descending(10**6)
print("S_10^6 (从大到小):", S_descending)


getcontext().prec = 28  # 设置精度

def significant_digits(number):
    return int(Decimal(str(number)).normalize().as_tuple().exponent * -1)

# 计算有效位数
sd_ascending = significant_digits(S_ascending)
sd_descending = significant_digits(S_descending)

print("有效位数 (从小到大):", sd_ascending)
print("有效位数 (从大到小):", sd_descending)


S_10^6 (从小到大): 7499990.000004939
S_10^6 (从大到小): 7499990.000004999
Significant digits (从小到大): 9
Significant digits (从大到小): 9


In [3]:
import numpy as np

def householder_vector(x):
    """计算Householder向量"""
    n = len(x)
    v = np.copy(x)
    sigma = np.sign(x[0]) * np.linalg.norm(x)
    v[0] = v[0] + sigma
    v = v / np.linalg.norm(v)
    return v

def hessenberg(A):
    """将矩阵转换为上Hessenberg形式"""
    n = A.shape[0]
    H = np.copy(A)
    Q = np.eye(n)
    
    for k in range(n-2):
        # 构造Householder变换
        x = H[k+1:, k]
        if np.linalg.norm(x) > 1e-15:
            v = householder_vector(x)
            
            # 应用Householder变换
            P = np.eye(n)
            P[k+1:, k+1:] -= 2.0 * np.outer(v, v)
            
            H = P @ H @ P
            Q = Q @ P
    
    return H, Q

def qr_iteration(H, Q, max_iter=100, tol=1e-10):
    """对Hessenberg矩阵进行QR迭代"""
    n = H.shape[0]
    T = np.copy(H)
    U = np.copy(Q)
    
    for iter in range(max_iter):
        # 计算QR分解
        mu = T[-1, -1]  # 位移
        Q_k, R = np.linalg.qr(T - mu * np.eye(n))
        T = R @ Q_k + mu * np.eye(n)
        U = U @ Q_k
        
        # 检查收敛性
        if np.all(np.abs(np.tril(T, -1)) < tol):
            break
    
    return T, U

def schur(A, max_iter=100, tol=1e-10):
    """计算矩阵的Schur分解"""
    # 步骤1：Hessenberg约简
    H, Q = hessenberg(A)
    
    # 步骤2：QR迭代
    T, U = qr_iteration(H, Q, max_iter, tol)
    
    return T, U

np.random.seed(42)
A = np.random.rand(4, 4)
#A = np.array([[5, 4, 2], [1, 3, 1], [2, 1, 4]], dtype=float)
# 计算Schur分解
T, U = schur(A)

# 验证结果
print("原始矩阵 A:")
print(A)
print("\nSchur三角形式 T:")
print(T)
print("\n酉矩阵 U:")
print(U)
print("\n验证 A = U @ T @ U.T:")
print(U @ T @ U.T)
print("\n误差:")
print(np.max(np.abs(A - U @ T @ U.T)))
print("验证：A = Q * T * Q^H")
print(np.allclose(A, U @ T @ U.T.conj(), atol=1e-5)) 

原始矩阵 A:
[[0.37454012 0.95071431 0.73199394 0.59865848]
 [0.15601864 0.15599452 0.05808361 0.86617615]
 [0.60111501 0.70807258 0.02058449 0.96990985]
 [0.83244264 0.21233911 0.18182497 0.18340451]]

Schur三角形式 T:
[[ 1.83720612e+00  2.15648150e-01 -5.00070353e-01 -7.33171871e-01]
 [-7.49791044e-59 -4.37670624e-01  5.71001314e-01  1.03084545e-01]
 [-2.45302048e-75 -5.37493519e-01 -3.50972035e-01  3.17630516e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -3.14039813e-01]]

酉矩阵 U:
[[ 0.64723702 -0.19937847  0.69394066 -0.24449709]
 [ 0.29851455 -0.49114585 -0.61124147 -0.54410356]
 [ 0.55766099 -0.14326941 -0.2922544   0.76359378]
 [ 0.4254263   0.83576184 -0.24375724 -0.24717811]]

验证 A = U @ T @ U.T:
[[0.37454012 0.95071431 0.73199394 0.59865848]
 [0.15601864 0.15599452 0.05808361 0.86617615]
 [0.60111501 0.70807258 0.02058449 0.96990985]
 [0.83244264 0.21233911 0.18182497 0.18340451]]

误差:
1.27675647831893e-15
验证：A = Q * T * Q^H
True


In [5]:
import numpy as np

def power_iteration(A, num_iterations=100, tolerance=1e-10):
    n = A.shape[0]
    v = np.random.rand(n)
    v = v / np.linalg.norm(v)
    
    for _ in range(num_iterations):
        v_new = A @ v
        v_new = v_new / np.linalg.norm(v_new)
        
        if np.allclose(v, v_new, rtol=tolerance):
            break
        v = v_new
    
    return v


def svd(A):
    m, n = A.shape
    k = min(m, n)
    
    U = np.zeros((m, k))
    S = np.zeros(k)
    VT = np.zeros((k, n))
    
    AT = A.T
    X = A.copy()
    
    for i in range(k):

        v = power_iteration(AT @ A)
        u = A @ v
        sigma = np.linalg.norm(u)
        u = u / sigma
        

        U[:, i] = u
        S[i] = sigma
        VT[i, :] = v
        

        outer_product = np.outer(u, v)
        X = X - sigma * outer_product
        A = X
        AT = A.T
    return U, S, VT

# 测试矩阵
A = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])


U, S, VT = svd(A)

# 验证结果
reconstructed = U @ np.diag(S) @ VT
print("原矩阵:\n", A)
print("\n重构矩阵:\n", reconstructed)

原矩阵:
 [[5. 4. 2.]
 [1. 3. 1.]
 [2. 1. 4.]]

重构矩阵:
 [[5. 4. 2.]
 [1. 3. 1.]
 [2. 1. 4.]]
